In [1]:
MODEL_DIR = None
MASK_CLS = 'ilm.mask.hierarchical.MaskHierarchical'

In [2]:
# Download pretrained model

if MODEL_DIR is None:
    !wget -nc --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1oYFLxkX6mWbmpEwQH8BmgE7iKix2W0zU' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1oYFLxkX6mWbmpEwQH8BmgE7iKix2W0zU" -O /tmp/ilm/eval_repro/models/sto_ilm/pytorch_model.bin && rm -rf /tmp/cookies.txt
    !wget -nc --no-check-certificate 'https://docs.google.com/uc?export=download&id=15JnXi7L6LeEB2fq4dFK2WRvDKyX46hVi' -O /tmp/ilm/eval_repro/models/sto_ilm/config.json
    MODEL_DIR = '/tmp/ilm/eval_repro/models/sto_ilm'

File ‘/tmp/ilm/eval_repro/models/sto_ilm/pytorch_model.bin’ already there; not retrieving.
File ‘/tmp/ilm/eval_repro/models/sto_ilm/config.json’ already there; not retrieving.


In [3]:
# Prepare tokenizer

import os
import pickle

import ilm.mask.util
import ilm.tokenize_util

# Create tokenizer
tokenizer = ilm.tokenize_util.Tokenizer.GPT2

# Update tokenizer with mask types
with open(os.path.join(MODEL_DIR, 'additional_ids_to_tokens.pkl'), 'rb') as f:
    additional_ids_to_tokens = pickle.load(f)
additional_tokens_to_ids = {v:k for k, v in additional_ids_to_tokens.items()}
print(additional_tokens_to_ids)
try:
    ilm.tokenize_util.update_tokenizer(additional_ids_to_tokens, tokenizer)
except ValueError:
    print('Already updated')

{'<|startofinfill|>': 50257, '<|endofinfill|>': 50258, '<|infill_document|>': 50259, '<|infill_paragraph|>': 50260, '<|infill_sentence|>': 50261, '<|infill_ngram|>': 50262, '<|infill_word|>': 50263}


In [4]:
# Load model

import torch
from transformers import GPT2LMHeadModel

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GPT2LMHeadModel.from_pretrained(MODEL_DIR)
model.eval()
_ = model.to(device)

In [30]:
# Create context

x = """
Math Class
Chris was bad at _. _ _ _ He ended up failing the test.
""".strip()

x_ids = ilm.tokenize_util.encode(x, tokenizer)

# Replace blanks with appropriate tokens from left to right
_blank_id = ilm.tokenize_util.encode(' _', tokenizer)[0]
x_ids[x_ids.index(_blank_id)] = additional_tokens_to_ids['<|infill_word|>']
x_ids[x_ids.index(_blank_id)] = additional_tokens_to_ids['<|infill_sentence|>']
x_ids[x_ids.index(_blank_id)] = additional_tokens_to_ids['<|infill_sentence|>']
x_ids[x_ids.index(_blank_id)] = additional_tokens_to_ids['<|infill_sentence|>']
print(ilm.tokenize_util.decode(x_ids, tokenizer))

Math Class
Chris was bad at<|infill_word|>.<|infill_sentence|><|infill_sentence|><|infill_sentence|> He ended up failing the test.


In [35]:
from infer import infill_with_ilm

generated = infill_with_ilm(
    model,
    additional_tokens_to_ids,
    x_ids)
for g in generated:
    print(ilm.tokenize_util.decode(g, tokenizer))

Math Class
Chris was bad at math. He started to fail the class. He became sad. He didn't want to disappoint his parents. He ended up failing the test.
